# Search Traffic Breakdown Metrics

[T301902](https://phabricator.wikimedia.org/T301902)

Pulling metrics for search traffic breakdown.

 - Total search volume per wiki: What is the total number of searches in the go bar?
 - Autocomplete only
 - Go bar-to-special:search volume per wiki:
   - What is the amount/% of searches initiated in the go-bar that end up on the special page?
   - What amount/percentage of queries that get redirected to special:search had no autocomplete suggestions?
   - What amount/percentage of queries that have no autocomplete suggestions also have zero full text search 
   results (i.e. 0 autosuggest suggestions > 0 special:search results)? inverse: what amount/percentage of queries with no autocomplete suggestions do have results in special:search?
 - Click through rates for Autocomplete searches and special searches

Including the following emerging languages for the search experimentations:

Priority 1:
Arabic, Bengali*, Spanish, Portuguese*, Russian

Priority 2: French*, Korean*, Indonesian, Ukrainian, Thai* ,Malaysian (?), Hindi, Tagalog, Afrikaans, Cantonese, Malayalam, Telugu

We pulled a reduced version of search event data from `searchsatisfaction` table in another notebook, and store in a new table `cchen_search.search_events`. 

In [1]:
from datetime import date, timedelta
import pandas as pd
import numpy as np

from wmfdata import hive, spark

You are using wmfdata v1.3.1, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [2]:
last_month = date.today().replace(day=1) - timedelta(days=1)
last_month_start = date.today().replace(day=1) - timedelta(days=last_month.day)

end_date = last_month.strftime("%Y-%m-%d")
start_date = last_month_start.strftime("%Y-%m-%d")

In [3]:
end_date

'2022-08-31'

In [4]:
start_date

'2022-08-01'

## Create Aggregation Tables

In [5]:
metrics_table = "cchen_search.special_search_metrics"

In [6]:
create_table_query = '''
CREATE TABLE IF NOT EXISTS {table_name} (
    log_date TIMESTAMP,
    wiki STRING,
    n_searches BIGINT,
    n_special_searches BIGINT,
    zero_auto_searches BIGINT,
    zero_auto_special_searches BIGINT,
    n_auto_clicks BIGINT,
    n_special_clicks BIGINT
)
'''

In [7]:
hive.run(create_table_query.format(
            table_name = metrics_table
))

## Get Metrics

In [6]:
search_metrics_query = '''

WITH search_events AS (
    SELECT * FROM cchen_search.search_event
    WHERE user_is_bot = false AND TO_DATE(dt) >= "{start_date}" AND TO_DATE(dt) <= "{end_date}"
), 
gobar_search AS (
    SELECT wiki, TO_DATE(dt) AS log_date, session_id, pageview_id
    FROM search_events
    WHERE action = "searchResultPage"
    AND source = "autocomplete"
    AND input_location LIKE "%header%"
), 
full_text AS (
    SELECT TO_DATE(dt) AS log_date, wiki, session_id, pageview_id, query, results_returned
    FROM search_events
    WHERE action = "searchResultPage"
    AND source = "fulltext"
), 
auto AS (
    SELECT TO_DATE(dt) AS log_date, wiki, session_id, pageview_id, query, results_returned
    FROM search_events
    WHERE action = "searchResultPage"
    AND source = "autocomplete"
    AND input_location LIKE "%header%"
), 
click AS (
    SELECT TO_DATE(dt) AS log_date, wiki, session_id, pageview_id
    FROM search_events
    WHERE action = "click"
    AND source = "autocomplete"
    AND click_position = -1

), 
visit AS (
    SELECT TO_DATE(dt) AS log_date, wiki, session_id, query
    FROM search_events
    WHERE action = "visitPage"
    AND source = "fulltext"
    AND click_position >= 0
), 
total_search AS (
    SELECT wiki, log_date,COUNT(DISTINCT session_id, pageview_id) AS n_searches
    FROM gobar_search
    GROUP BY wiki, log_date
), 
special_search AS (
   SELECT a.log_date, 
       a.wiki,
       COUNT(DISTINCT a.session_id, a.pageview_id) AS n_special_searches,
       COUNT(DISTINCT(CASE WHEN a.results_returned = 0 THEN (a.session_id, a.pageview_id) END)) AS zero_auto_searches,
       COUNT(DISTINCT(CASE WHEN a.results_returned = 0 AND f.results_returned IS NULL THEN (a.session_id, a.pageview_id) END)) AS zero_auto_special_searches
   FROM full_text f 
     INNER JOIN auto a ON (f.session_id = a.session_id AND f.query = a.query AND f.log_date = a.log_date AND f.wiki = a.wiki)
     INNER JOIN click c ON (a.session_id = c.session_id AND a.pageview_id = c.pageview_id AND a.log_date = c.log_date AND a.wiki = c.wiki)
   WHERE f.pageview_id != a.pageview_id
   GROUP BY a.log_date, a.wikI
), 
gobar_visit AS (
    SELECT 
        gs.log_date,
        s.wiki,
        COUNT(DISTINCT s.session_id, s.pageview_id) AS n_clicks
    FROM search_events s
    INNER JOIN gobar_search gs 
    ON (s.wiki = gs.wiki AND gs.log_date = TO_DATE(s.dt) AND s.session_id = gs.session_id AND s.pageview_id = gs.pageview_id)
    AND action = "click"
    AND source = "autocomplete"
    AND s.click_position >= 0
    GROUP BY gs.log_date,s.wiki
),  
special_visit AS (
    SELECT
      a.log_date, 
      a.wiki,
      COUNT(DISTINCT a.session_id, a.pageview_id) AS n_special_clicks
    FROM full_text f 
      INNER JOIN auto a ON (f.session_id = a.session_id AND f.query = a.query AND f.log_date = a.log_date AND f.wiki = a.wiki)
      INNER JOIN click c ON (a.session_id = c.session_id AND a.pageview_id = c.pageview_id AND a.log_date = c.log_date AND a.wiki = c.wiki)
      INNER JOIN visit v ON (f.session_id = v.session_id AND f.query = v.query AND f.log_date = v.log_date AND f.wiki = v.wiki)
    WHERE f.pageview_id != a.pageview_id
    GROUP BY a.log_date, a.wiki
)

INSERT INTO {table}
SELECT
    ts.log_date,
    ts.wiki,
    SUM(n_searches) AS n_searches,
    SUM(n_special_searches) AS n_special_searches,
    SUM(zero_auto_searches) AS zero_auto_searches,
    SUM(zero_auto_special_searches) AS zero_auto_special_searches,
    SUM(n_clicks) AS n_auto_clicks,
    SUM(n_special_clicks) AS n_special_clicks
FROM total_search ts
  INNER JOIN special_search ss ON (ts.log_date = ss.log_date AND ts.wiki = ss.wiki)
  INNER JOIN gobar_visit gc ON (gc.log_date = ss.log_date AND gc.wiki = ss.wiki)
  INNER JOIN special_visit sc ON (gc.log_date = sc.log_date AND gc.wiki = sc.wiki)
GROUP BY ts.log_date, ts.wiki
'''

In [ ]:
hive.run(search_metrics_query.format(
                            start_date = start_date,
                            end_date = end_date,
                            table = metrics_table
                          ))